In [2]:
import nltk
from nltk.tag.stanford import StanfordNERTagger
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize
from nltk.chunk import conlltags2tree, tree2conlltags

sw = stopwords.words("english")

path_to_model = r'C:\Users\inves\AppData\Local\Programs\Python\Python35\Lib\site-packages\nltk\stanford-ner-2018-02-27\classifiers\english.all.3class.distsim.crf.ser.gz'
path_to_jar = r'C:\Users\inves\AppData\Local\Programs\Python\Python35\Lib\site-packages\nltk\stanford-ner-2018-02-27\stanford-ner.jar'
tagger = StanfordNERTagger(path_to_model, path_to_jar = path_to_jar)

In [3]:
from gensim.models.word2vec import LineSentence
from gensim.models import Phrases
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

#visualization libraries
import pyLDAvis
import pyLDAvis.gensim

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
import os
import pandas as pd
import numpy as np
import codecs
import itertools as it
from bs4 import BeautifulSoup
import warnings
import pickle
from collections import Counter
import re
import datetime
import string
import time

java_path = 'C:/Program Files/Java/jdk-10.0.1/bin/java.exe'
os.environ['JAVAHOME'] = java_path

In [5]:
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\inves\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\inves\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\inves\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [ ]:
docket_original = list(new_df['Original Docket Text'])

In [ ]:
output = []
for i in range(5):
    org_str = []
    name_str = []
    stripped_str1 = []
    stripped_str2 = []

    tokens = word_tokenize(docket_original[i])
    for SFlabel, NLlabel, token in zip(tagger.tag(tokens), tree2conlltags(ne_chunk(pos_tag(tokens))), tokens):
        print(SFlabel, NLlabel, token)
        if SFlabel[1] == 'ORGANIZATION':
            org_str.append(SFlabel[0])
            stripped_str1.append('-ORG-')
        elif SFlabel[1] == 'PERSON':
            name_str.append(SFlabel[0])
            stripped_str1.append('-NAME-')
        else:
            stripped_str1.append(token)
            stripped_str2.append(token)

    output.append([docket_original[i],
                   ' '.join(org_str),
                   ' '.join(name_str),
                   ' '.join(stripped_str1),
                   ' '.join(stripped_str2)])
    
for i in range(5):
    print('docket text:', i)
    print(output[i], '\n')

### We may want to start here:

In [6]:
filename = 'docket_texts/train/DT/basic_df.pickle'
#to load
with open(filename, 'rb') as handle:
    NER_df = pickle.load(handle)
    
new_df = NER_df.copy()
print(new_df.shape)
new_df.head()

(3244, 5)


,Original Docket Text,Organization Portion,Name Portion,Identifying Org and Name,Stripped Org and Name
0,"COMPLAINT against Cardiogenics Holdings, Inc. ...","Cardiogenics Holdings , Inc. LG Capital Funding",( Steinmetz Michael Bowens Priscilla,COMPLAINT against -ORG- -ORG- -ORG- -ORG- fili...,"COMPLAINT against filing fee $ 400 , receipt n..."
1,Case assigned to Judge Ann M Donnelly and Magi...,Individual Practices of the assigned Judges,Ann M Donnelly Vera M. Scanlon Bowens Priscilla,Case assigned to Judge -NAME- -NAME- -NAME- an...,Case assigned to Judge and Magistrate Judge . ...
2,"Summons Issued as to Cardiogenics Holdings, In...",Cardiogenics Holdings,Bowens Priscilla,"Summons Issued as to -ORG- -ORG- , Inc.. ( -NA...","Summons Issued as to , Inc.. ( , ) ( Entered :..."
3,NOTICE - emailed attorney regarding missing se...,,Bowens Priscilla,NOTICE - emailed attorney regarding missing se...,NOTICE - emailed attorney regarding missing se...
4,In accordance with Rule 73 of the Federal Rule...,,Bowens Priscilla,In accordance with Rule 73 of the Federal Rule...,In accordance with Rule 73 of the Federal Rule...


In [25]:
### Normalize
def normalize(docket_original):

    url_regex1 = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    url_regex2 = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    date_regex = '(\d{1,2}[\/ ](\d{2}|January|Jan|February|Feb|March|Mar|April|Apr|May|May|June|Jun|July|Jul|August|Aug|September|Sep|October|Oct|November|Nov|December|Dec)[\/ ]\d{2,4})'
    punct_regex = r"[^a-zA-Z0-9]"
    num_regex = "\d+"
    extraspace_regex = " +"
    docket_original = docket_original.split(' ')
    docket_normalized = [text.lower() for text in docket_original]
    docket_nourl1 = [re.sub(url_regex1, "url", text) for text in docket_normalized]
    docket_nourl2 = [re.sub(url_regex2, "url", text) for text in docket_nourl1]
    docket_nodate = [re.sub(date_regex, "date", text) for text in docket_nourl2]
    docket_nopunct = [re.sub(punct_regex, " ", text) for text in docket_nodate]
    docket_nonum = [re.sub(num_regex, " ", text) for text in docket_nopunct]
    #new, remove if the last words are entered date
    docket_noextraspace = re.sub(extraspace_regex, " ", ' '.join(docket_nonum))
    docket_no_entereddate = docket_noextraspace.replace('entered date', '')
    
    return docket_no_entereddate

In [26]:
t0 = time.time()
new_df['normalized'] = new_df['Stripped Org and Name'].apply(lambda x: normalize(x))
print('took {} seconds'.format(time.time() - t0))

took 1.380718469619751 seconds


In [27]:
new_df['normalized'].iloc[0]

'complaint against filing fee receipt number was the disclosure statement on civil cover sheet completed yes filed by llc additional attachment s added on date civil cover sheet proposed summons  '

In [28]:
def tokenize(docket_noextraspace):
    return word_tokenize(docket_noextraspace)

In [29]:
t0 = time.time()
new_df['tokenize'] = new_df['normalized'].apply(tokenize)
print('took {} seconds'.format(time.time() - t0))

took 0.832021951675415 seconds


In [30]:
def stopwords(docket_tokenized):  
    return [word for word in docket_tokenized if word not in sw]

In [31]:
t0 = time.time()
new_df['nostop'] = new_df['tokenize'].apply(stopwords)
print('took {} seconds'.format(time.time() - t0))

took 0.268829345703125 seconds


In [32]:
#this is updated, comparing mannual_topics_20180828 and manual_topics_20180911 and manual_topics_20180919
manual_topics_df = pd.read_csv('manual_topics_20180919.csv') 
manual_topics_df = manual_topics_df.apply(lambda x: x.astype(str).str.lower())
manual_topics_dict = manual_topics_df.to_dict('list')
for topic in manual_topics_dict.keys():
    manual_topics_dict[topic] = [keyword for keyword in manual_topics_dict[topic] if keyword != 'nan']

In [33]:
#this is updated, comparing mannual_topics_20180828 and manual_topics_20180911 and manual_topics_20180919
manual_topics_df = pd.read_csv('manual_topics_20180919.csv') 
manual_topics_df = manual_topics_df.apply(lambda x: x.astype(str).str.lower())
manual_topics_dict = manual_topics_df.to_dict('list')
for topic in manual_topics_dict.keys():
    manual_topics_dict[topic] = [keyword for keyword in manual_topics_dict[topic] if keyword != 'nan']

In [34]:
#output a list of topics
def mannual_topic_assignment(text):
    #text = text.split()
    #print(text)
    output = []
    for topic in manual_topics_dict.keys():
        for keyword in manual_topics_dict[topic]:
            if ' '.join(text).find(keyword) != -1:
                output.append(topic)
    #print(output)
    #if there's motion to dismiss, then remove noaction
    if ' '.join(text).find('motion dismiss') != -1 and 'NoAction' in output:
        output = list(set(output))
        output.remove('NoAction')
    return ', '.join(set(output))

In [35]:
docket_texts_output = list(new_df['nostop'])

docket_texts_output_DT = []
topics_DT = []
NoAction = []

for text in docket_texts_output:
    topic = mannual_topic_assignment(text)
    #print(text, topic)
    if topic != '':
        docket_texts_output_DT.append('')
        topics_DT.append(topic)
        if 'NoAction' in topic:
            NoAction.append(1)
        else:
            NoAction.append(0)
    else:
        docket_texts_output_DT.append(text)
        topics_DT.append('')
        NoAction.append(0)

In [36]:
new_df['DT Topics'] = pd.Series(topics_DT)
new_df['NoAction'] = pd.Series(NoAction)
new_df['Removed unnecessary POS & vocab DT'] = pd.Series(docket_texts_output_DT)
print(new_df.shape)
new_df.head()

(3244, 11)


,Original Docket Text,Organization Portion,Name Portion,Identifying Org and Name,Stripped Org and Name,normalized,tokenize,nostop,DT Topics,NoAction,Removed unnecessary POS & vocab DT
0,"COMPLAINT against Cardiogenics Holdings, Inc. ...","Cardiogenics Holdings , Inc. LG Capital Funding",( Steinmetz Michael Bowens Priscilla,COMPLAINT against -ORG- -ORG- -ORG- -ORG- fili...,"COMPLAINT against filing fee $ 400 , receipt n...",complaint against filing fee receipt number wa...,"[complaint, against, filing, fee, receipt, num...","[complaint, filing, fee, receipt, number, disc...","Complaints, Service of Process, Motions",0,
1,Case assigned to Judge Ann M Donnelly and Magi...,Individual Practices of the assigned Judges,Ann M Donnelly Vera M. Scanlon Bowens Priscilla,Case assigned to Judge -NAME- -NAME- -NAME- an...,Case assigned to Judge and Magistrate Judge . ...,case assigned to judge and magistrate judge pl...,"[case, assigned, to, judge, and, magistrate, j...","[case, assigned, judge, magistrate, judge, ple...",NoAction,1,
2,"Summons Issued as to Cardiogenics Holdings, In...",Cardiogenics Holdings,Bowens Priscilla,"Summons Issued as to -ORG- -ORG- , Inc.. ( -NA...","Summons Issued as to , Inc.. ( , ) ( Entered :...",summons issued as to inc,"[summons, issued, as, to, inc]","[summons, issued, inc]",Service of Process,0,
3,NOTICE - emailed attorney regarding missing se...,,Bowens Priscilla,NOTICE - emailed attorney regarding missing se...,NOTICE - emailed attorney regarding missing se...,notice emailed attorney regarding missing seco...,"[notice, emailed, attorney, regarding, missing...","[notice, emailed, attorney, regarding, missing...","NoAction, Notices",1,
4,In accordance with Rule 73 of the Federal Rule...,,Bowens Priscilla,In accordance with Rule 73 of the Federal Rule...,In accordance with Rule 73 of the Federal Rule...,in accordance with rule of the federal rules o...,"[in, accordance, with, rule, of, the, federal,...","[accordance, rule, federal, rules, civil, proc...","Order, Notices, Motions, Judgment",0,


In [37]:
def lemm(docket_nostop):
    return ' '.join([WordNetLemmatizer().lemmatize(word, pos='v') for word in docket_nostop])

In [38]:
t0 = time.time()
new_df['lemmed'] = new_df['Removed unnecessary POS & vocab DT'].apply(lemm)
print('took {} seconds'.format(time.time() - t0))
new_df.shape

took 1.985579013824463 seconds


(3244, 12)

In [39]:
docket_phrase2 = list(new_df['lemmed'])
docket_phrase2[:5]

['', '', '', '', '']

In [40]:
unigram_sentences_filepath = 'docket_texts/train/DT/unigram_nltk_newsop2.txt'

In [41]:
%%time
# turn the lemmatized corpus into unigram sentences
with codecs.open(unigram_sentences_filepath, 'w', encoding = 'utf_8') as f:
    for sentence in docket_phrase2:
        f.write(sentence + '\n')

Wall time: 18.9 ms


In [42]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [43]:
bigram_model_filepath = 'docket_texts/train/DT/bigram_model_newsop2' 

In [44]:
%%time

# store our bigram model
bigram_model = Phrases(unigram_sentences)
bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk if we don't want to run this again
#bigram_model = Phrases.load(bigram_model_filepath)

Wall time: 128 ms


In [45]:
bigram_sentences_filepath = 'docket_texts/train/DT/bigram_sentences_newsop2.txt'

In [46]:
%%time

# apply the bigram model, and write it to file
with codecs.open(bigram_sentences_filepath, 'w', encoding = 'utf_8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = ' '.join(bigram_model[unigram_sentence])
        f.write(bigram_sentence + '\n')

Wall time: 36.9 ms


c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [47]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [48]:
print('\nUnigram sentence:')
for unigram_sentence in it.islice(unigram_sentences, 0, 10):
    print(' '.join(unigram_sentence))
print('\nBigram sentence:')
for bigram_sentence in it.islice(bigram_sentences, 0, 10):
    print(' '.join(bigram_sentence))


Unigram sentence:
status report llc kehrli
corporate disclosure statement
status report propose brief schedule llc kehrli
status report llc kehrli
propose find fact llc kehrli
witness list llc kehrli
status report llc kehrli
status report llc kehrli
rule statement file llc
civil cover sheet file

Bigram sentence:
status_report llc_kehrli
corporate disclosure statement
status_report propose brief schedule llc_kehrli
status_report llc_kehrli
propose_find fact llc_kehrli
witness list llc_kehrli
status_report llc_kehrli
status_report llc_kehrli
rule statement file llc
civil_cover sheet file


In [49]:
trigram_model_filepath = 'docket_texts/train/DT/trigram_model_newsop2'

In [57]:
%%time

# again, using Phrases to attach more words to phrases already formed
trigram_model = Phrases(bigram_sentences)
trigram_model.save(trigram_model_filepath)

# load the finished model from disk
#trigram_model = Phrases.load(trigram_model_filepath)

Wall time: 7.98 ms


In [58]:
trigram_sentences_filepath = 'docket_texts/train/DT/trigram_sentences_newsop2.txt'

In [59]:
%%time

with codecs.open(trigram_sentences_filepath, 'w', encoding = 'utf_8') as f:
    for bigram_sentence in bigram_sentences:
        #print('Bi', bigram_sentence)
        trigram_sentence = ' '.join(trigram_model[bigram_sentence])
        #print('Tri', trigram_sentence)
        f.write(trigram_sentence + '\n')

Wall time: 8.95 ms


c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [60]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [61]:
start = 0
finish = 5
print('Original text:')
print(docket_phrase2[start:finish], '\n')

print('\nUNIGRAM Sentence:')
for unigram_sentence in it.islice(unigram_sentences, start, finish):
    print(' '.join(unigram_sentence))
print('\nBIGRAM Sentence:')
for bigram_sentence in it.islice(bigram_sentences, start, finish):
    print(' '.join(bigram_sentence))
print('\nTRIGRAM Sentence:')
for trigram_sentence in it.islice(trigram_sentences, start, finish):
    print(' '.join(trigram_sentence))

Original text:
['', '', '', '', ''] 


UNIGRAM Sentence:
status report llc kehrli
corporate disclosure statement
status report propose brief schedule llc kehrli
status report llc kehrli
propose find fact llc kehrli

BIGRAM Sentence:
status_report llc_kehrli
corporate disclosure statement
status_report propose brief schedule llc_kehrli
status_report llc_kehrli
propose_find fact llc_kehrli

TRIGRAM Sentence:
status_report llc_kehrli
corporate_disclosure statement
status_report propose brief schedule llc_kehrli
status_report llc_kehrli
propose_find fact llc_kehrli


In [62]:
def trigram_transform(texts):
    display = False
    texts = str(texts)
    trigram_output = ''
    #print(texts)

    remove_trigram = ['calendar_day', 'court_notice_intend', 'minute_entry_proceeding_hold', 'court_reportertranscriber_abovecaptioned_matter',
                      'redaction_calendar_day', 'rule_statement', 'obtain_pacer', 'may_obtain_pacer', 'reportertranscriber_abovecaptioned_matter',
                      'redact_transcript_deadline', 'send_chamber', "official_transcript_notice_give", "notice_intent_request", "proceed_hold", 
                      "fee_receipt_number", "civil_procedure", "pursuant_frcp", "official_transcript_conference", 
                      "purchase_reportertranscriber_deadline_release", "et_al", "mail_chamber", "transcript_restriction", "redaction_transcript", 
                      "transcript_view_public_terminal", "transcript_make_remotely", "associated_et_al", "electronically_available_public_without", 
                      "genesys_id", "release_transcript_restriction", "adar_bay", "redaction_request_due", "new_york", "official_transcript_conference", 
                      "transcript_make_remotely", "transcript_proceeding_conference_hold", "redaction_transcript",
                      'affidavit_jr._c.p.a', 'corporate_parent', 'certain_underwriter', 'federal_rule_civil_procedure', 'redaction_request', 
                      'official_transcript', 'rule_disclosure', 'rule_corporate_disclosure', 'place_vault', 'public_without_redaction_calendar', 
                      'purchase_deadline_release_transcript', 'transcript_proceeding_hold', 'transcript_remotely_electronically_available',
                      'minute_entry_hold', 'discovery_hear_hold', 'jury_trial_hold', "sign_judge",'place_vault']

    if texts == None:
        return None
    
    unigram_review = []
    for word in texts.split():
        unigram_review.append(word)
    if display:
        print('Uni: ', unigram_review)
    bigram_review = bigram_model[unigram_review]
    if display:
        print('Bi: ', bigram_review)
    trigram_review = trigram_model[bigram_review]
    if display:
        print('Tri: ', trigram_review)
    trigram_review = [phrase for phrase in trigram_review if phrase not in remove_trigram]
    if display:
        print('Tri removed: ', trigram_review)
    trigram_output += ' '.join(trigram_review)
    
    return trigram_output

In [63]:
docket_phrase3 = [trigram_transform(text) for text in docket_phrase2]

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [64]:
print(docket_phrase2[:5])
print(docket_phrase3[:5])
len(set(docket_phrase3))

['', '', '', '', '']
['', '', '', '', '']


122

In [65]:
new_df['phrases'] = pd.Series(docket_phrase3)
new_df.shape

(3244, 13)

In [66]:
new_df.columns

Index(['Original Docket Text', 'Organization Portion', 'Name Portion',
       'Identifying Org and Name', 'Stripped Org and Name', 'normalized',
       'tokenize', 'nostop', 'DT Topics', 'NoAction',
       'Removed unnecessary POS & vocab DT', 'lemmed', 'phrases'],
      dtype='object')

In [67]:
i = 1
print('orignal text: ')
print(new_df['Original Docket Text'].iloc[i], '\n')
print('org and name removed: ')
print(new_df['Stripped Org and Name'].iloc[i], '\n')
print('normalized: ')
print(new_df['normalized'].iloc[i], '\n')
print('stopwords removed: ')
print(' '.join(new_df['nostop'].iloc[i]), '\n')
print('after lemmetization: ')
print(new_df['lemmed'].iloc[i], '\n')
print('after phrase modeling: ')
print(new_df['phrases'].iloc[i], '\n')

orignal text: 
Case assigned to Judge Ann M Donnelly and Magistrate Judge Vera M. Scanlon. Please download and review the Individual Practices of the assigned Judges, located on our website. Attorneys are responsible for providing courtesy copies to judges where their Individual Practices require such. (Bowens, Priscilla) (Entered: 03/11/2016) 

org and name removed: 
Case assigned to Judge and Magistrate Judge . Please download and review the , located on our website . Attorneys are responsible for providing courtesy copies to judges where their Individual Practices require such . ( , ) ( Entered : 03/11/2016 ) 

normalized: 
case assigned to judge and magistrate judge please download and review the located on our website attorneys are responsible for providing courtesy copies to judges where their individual practices require such   

stopwords removed: 
case assigned judge magistrate judge please download review located website attorneys responsible providing courtesy copies judges 

In [68]:
remove_pos = list(pd.read_excel('NLP_to_be_removed.xlsx', sheetname = 0, header = None)[0])
remove_word = list(pd.read_excel('NLP_to_be_removed.xlsx', sheetname = 1, header = None)[0])
remove_trigram = list(pd.read_excel('NLP_to_be_removed.xlsx', sheetname = 2, header = None)[0])

In [69]:
def remove_unnecessary(text):
    new_text = []
    text = text.split(' ')
    for word in text:
        if word not in remove_word and word not in remove_trigram:
            new_text.append(word)
    return ' '.join(new_text)

In [70]:
new_df['cleaned'] = new_df['phrases'].apply(remove_unnecessary)
print(new_df.shape)
new_df.head()

(3244, 14)


,Original Docket Text,Organization Portion,Name Portion,Identifying Org and Name,Stripped Org and Name,normalized,tokenize,nostop,DT Topics,NoAction,Removed unnecessary POS & vocab DT,lemmed,phrases,cleaned
0,"COMPLAINT against Cardiogenics Holdings, Inc. ...","Cardiogenics Holdings , Inc. LG Capital Funding",( Steinmetz Michael Bowens Priscilla,COMPLAINT against -ORG- -ORG- -ORG- -ORG- fili...,"COMPLAINT against filing fee $ 400 , receipt n...",complaint against filing fee receipt number wa...,"[complaint, against, filing, fee, receipt, num...","[complaint, filing, fee, receipt, number, disc...","Complaints, Service of Process, Motions",0,,,,
1,Case assigned to Judge Ann M Donnelly and Magi...,Individual Practices of the assigned Judges,Ann M Donnelly Vera M. Scanlon Bowens Priscilla,Case assigned to Judge -NAME- -NAME- -NAME- an...,Case assigned to Judge and Magistrate Judge . ...,case assigned to judge and magistrate judge pl...,"[case, assigned, to, judge, and, magistrate, j...","[case, assigned, judge, magistrate, judge, ple...",NoAction,1,,,,
2,"Summons Issued as to Cardiogenics Holdings, In...",Cardiogenics Holdings,Bowens Priscilla,"Summons Issued as to -ORG- -ORG- , Inc.. ( -NA...","Summons Issued as to , Inc.. ( , ) ( Entered :...",summons issued as to inc,"[summons, issued, as, to, inc]","[summons, issued, inc]",Service of Process,0,,,,
3,NOTICE - emailed attorney regarding missing se...,,Bowens Priscilla,NOTICE - emailed attorney regarding missing se...,NOTICE - emailed attorney regarding missing se...,notice emailed attorney regarding missing seco...,"[notice, emailed, attorney, regarding, missing...","[notice, emailed, attorney, regarding, missing...","NoAction, Notices",1,,,,
4,In accordance with Rule 73 of the Federal Rule...,,Bowens Priscilla,In accordance with Rule 73 of the Federal Rule...,In accordance with Rule 73 of the Federal Rule...,in accordance with rule of the federal rules o...,"[in, accordance, with, rule, of, the, federal,...","[accordance, rule, federal, rules, civil, proc...","Order, Notices, Motions, Judgment",0,,,,


### If you are happy with the NLP process, START/STOP here

### If you need to continue to merge with domain expert's answer key for the traning set, proceed:

In [76]:
train_filename = r'E:\WinUser\Documents\Python Code\AI Paralegal\docket_texts\Train\DT\train_tempfile.xlsx'
old_df = pd.read_excel(train_filename)

print(old_df.shape)
old_df.tail(20)

(3203, 10)


,Original Docket Text,normalized,lemmed,phrases,DT Topics,NoAction,Action,If Action,Action revised,If Action revised
3183,SEALED DOCUMENT placed in vault.(nm) (Entered:...,sealed document placed in vault nm entered date,NaN,NaN,"NoAction, Motions",1,N,NaN,N,NaN
3184,Please be advised that Judge Abrams Courtroom ...,please be advised that judge abrams courtroom ...,please advise judge abrams courtroom chamber m...,please advise judge abrams courtroom chamber m...,NaN,0,Y,Attorney; Assistant,Y,Attorney; Assistant
3185,MOTION for Andrew John Estes and Simona Gory t...,motion for and to withdraw as attorney for and...,NaN,NaN,Motions,0,Y,Attorney,Y,Attorney
3186,MEMO ENDORSEMENT granting 204 MOTION FOR LEAVE...,memo endorsement granting motion for leave to ...,NaN,NaN,"Motions, Order",0,Y,Attorney,Y,Attorney
3187,SIXTH REVISED SCHEDULING ORDER: Expert Discove...,sixth revised scheduling order expert discover...,NaN,NaN,"Other Answers, Discovery, Order, Motions",0,Y,Attorney; Assistant,Y,Attorney; Assistant
3188,ENDORSED LETTER addressed to Magistrate Judge ...,endorsed letter addressed to magistrate judge ...,NaN,NaN,"Discovery, Notices, Service of Process, Letter...",0,Y,Attorney; Assistant,Y,Attorney; Assistant
3189,ENDORSED LETTER addressed to Magistrate Judge ...,endorsed letter addressed to magistrate judge ...,NaN,NaN,"Discovery, Notices, Service of Process, Letter...",0,Y,Attorney; Assistant,Y,Attorney; Assistant
3190,MOTION for Randall T. Adams to Withdraw as Att...,motion for to withdraw as attorney for plainti...,NaN,NaN,Motions,0,Y,Attorney,Y,Attorney
3191,MEMO ENDORSEMENT ON MOTION FOR LEAVE TO WITHDR...,memo endorsement on motion for leave to withdr...,NaN,NaN,Motions,0,Y,Attorney,Y,Attorney
3192,MOTION for Joseph P. Fishman to Withdraw as At...,motion for to withdraw as attorney document fi...,NaN,NaN,"Motions, Service of Process",0,Y,Attorney; Assistant,Y,Attorney; Assistant


In [85]:
print(old_df.drop_duplicates().shape)
old_df.columns

(3203, 10)


Index(['Original Docket Text', 'normalized', 'lemmed', 'phrases', 'DT Topics',
       'NoAction', 'Action', 'If Action', 'Action revised',
       'If Action revised'],
      dtype='object')

In [82]:
print(new_df[['Original Docket Text', 'normalized', 'lemmed', 'phrases', 'DT Topics']].drop_duplicates().shape)
new_df.columns

(3203, 5)


Index(['Original Docket Text', 'Organization Portion', 'Name Portion',
       'Identifying Org and Name', 'Stripped Org and Name', 'normalized',
       'tokenize', 'nostop', 'DT Topics', 'NoAction',
       'Removed unnecessary POS & vocab DT', 'lemmed', 'phrases', 'cleaned'],
      dtype='object')

In [87]:
newer_df = new_df[['Original Docket Text', 'normalized', 'lemmed', 'phrases', 'DT Topics']].drop_duplicates().merge(old_df[['Original Docket Text', 'NoAction', 'Action', 'If Action', 'Action revised', 'If Action revised']], 
                                                                                                                    on = 'Original Docket Text', how = 'left', indicator = True)
print(newer_df.shape)
newer_df[newer_df['_merge'] != 'both'].head()

(3203, 11)


,Original Docket Text,normalized,lemmed,phrases,DT Topics,NoAction,Action,If Action,Action revised,If Action revised,_merge
2319,Appeal Record Sent to USCA (Electronic File). ...,appeal record sent to usca electronic file cer...,,,"Complaints, Letter and Response, Other Answers...",NaN,NaN,NaN,NaN,NaN,left_only


In [104]:
old_df.iloc[2319]

Original Docket Text    Appeal Record Sent to USCA (Electronic File). ...
normalized              appeal record sent to usca electronic file cer...
lemmed                                                                NaN
phrases                                                               NaN
DT Topics               Notices, Order, Letter and Response, Judgment,...
NoAction                                                                0
Action                                                                NaN
If Action                                                             NaN
Action revised                                                        NaN
If Action revised                                                     NaN
Name: 2319, dtype: object

In [102]:
new_df['Original Docket Text'].apply(len).sort_values(ascending = False)
new_df.iloc[2352]

Original Docket Text                  Appeal Record Sent to USCA (Electronic File). ...
Organization Portion                  World Trade Company Silverstein Development Co...
Name Portion                          Francis M. Lynch Swanke Hayden Connell Archete...
Identifying Org and Name              Appeal Record Sent to USCA ( Electronic File )...
Stripped Org and Name                 Appeal Record Sent to USCA ( Electronic File )...
normalized                            appeal record sent to usca electronic file cer...
tokenize                              [appeal, record, sent, to, usca, electronic, f...
nostop                                [appeal, record, sent, usca, electronic, file,...
DT Topics                             Complaints, Letter and Response, Other Answers...
NoAction                                                                              0
Removed unnecessary POS & vocab DT                                                     
lemmed                          

In [105]:
print(new_df['DT Topics'].iloc[2352])
print(old_df['DT Topics'].iloc[2319])

Complaints, Letter and Response, Other Answers, Service of Process, Judgment, Order, Notices, Answers to Complaints, Motions
Notices, Order, Letter and Response, Judgment, Answers to Complaints, Other Answers, Complaints, Motions, Service of Process


In [ ]:
new_df.iloc[2352]

In [72]:
filename = 'docket_texts/train/DT/basic_cleaned_with_labels.pickle'

In [73]:
#to save
with open(filename, 'wb') as handle: 
    pickle.dump(new_df, handle, protocol = pickle.HIGHEST_PROTOCOL)

### If you need domain expert to check your work, continue. Otherwise, STOP here

In [ ]:
new_df[['Original Docket Text', 'normalized', 'DT Topics', 'lemmed', 'phrases']].drop_duplicates().to_csv('Check if keywords working.csv', index = False)

In [ ]:
print('shape before dedupe: {}'.format(new_df.shape))
print('shape after dedupe: {}'.format(new_df[['Original Docket Text', 'normalized', 'DT Topics', 'lemmed', 'phrases']].drop_duplicates().shape))

In [ ]:
chris_file = pd.read_excel(r'temporary file - Sept 19.xlsx')
print(chris_file.shape)
chris_file.tail(1)

In [ ]:
another_df = new_df[['Original Docket Text', 'normalized', 'lemmed', 'phrases', 'DT Topics', 'NoAction']].drop_duplicates().merge(chris_file[['Original Docket Text', 'Action', 'If Action']], on = 'Original Docket Text', how = 'left')
print(another_df.shape)
another_df.tail(1)
#another_df.to_excel('tempfile.xlsx', index = False)

In [ ]:
another_df['Action revised'] = np.where(another_df['NoAction'] == 1, 'N', another_df['Action'])
another_df['If Action revised'] = np.where(another_df['NoAction'] == 1, '', another_df['If Action'])

another_df.head()

In [ ]:
#export to check
another_df.to_excel('tempfile.xlsx', index = False)
#applying methodology to the testing file is at another notebook